In [2]:
import pandas as pd
import numpy as np

In [27]:
old = pd.read_parquet('export/gr-work-genders.parquet')
old.head()

,work_id,cluster,gender
0,40.0,1234122.0,male
1,40.0,NaN,no-book
2,62.0,3746546.0,female
3,81.0,5581154.0,female
4,81.0,NaN,no-book


In [28]:
old = old[old.work_id.notnull()]
old['work_id'] = old.work_id.astype('i4')
old.set_index('work_id', inplace=True)
old.sort_index(inplace=True)
old.head()

,cluster,gender
work_id,,
40,1234122.0,male
40,NaN,no-book
62,3746546.0,female
81,5581154.0,female
81,NaN,no-book


In [29]:
old['gender'] = old['gender'].str.replace('-loc', '-book')
old['gender'] = old['gender'].str.replace('-viaf-author', '-author-rec')

In [8]:
new = pd.read_parquet('../bd-df/goodreads/gr-work-gender.parquet')
new = new[new.work_id.notnull()]
new['work_id'] = new.work_id.astype('i4')
new.set_index('work_id', inplace=True)
new.sort_index(inplace=True)
new.head()

,cluster,gender
work_id,,
40,103685309,male
62,101077477,female
81,106806269,female
84,102279759,female
87,109405824,female


In [30]:
old.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1702019 entries, 40 to 58379481
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   cluster  float64
 1   gender   object 
dtypes: float64(1), object(1)
memory usage: 39.0+ MB


In [10]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1521962 entries, 40 to 58379481
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   cluster  1521962 non-null  int32 
 1   gender   1521962 non-null  object
dtypes: int32(1), object(1)
memory usage: 29.0+ MB


In [31]:
both = old.join(new, how='outer', lsuffix='_old', rsuffix='_new')
both.head()

,cluster_old,gender_old,cluster_new,gender_new
work_id,,,,
40,1234122.0,male,103685309,male
40,NaN,no-book,103685309,male
62,3746546.0,female,101077477,female
81,5581154.0,female,106806269,female
81,NaN,no-book,106806269,female


In [32]:
mm = both[both['gender_old'] != both['gender_new']]
mm

,cluster_old,gender_old,cluster_new,gender_new
work_id,,,,
40,NaN,no-book,103685309,male
81,NaN,no-book,106806269,female
84,NaN,no-book,102279759,female
87,NaN,no-book,109405824,female
98,NaN,no-book,111175964,female
...,...,...,...,...
58370103,NaN,no-book,458370103,no-book-author
58376302,NaN,no-book,458376302,no-book-author
58376303,NaN,no-book,458376303,no-book-author


In [35]:
mm.loc[mm.cluster_old.isnull(), 'gender_new'].value_counts()

no-book-author    498188
male               42309
female             36305
unknown            10158
ambiguous           3244
no-author-rec       2664
Name: gender_new, dtype: int64

In [37]:
mm[mm.cluster_old.isnull()]

,cluster_old,gender_old,cluster_new,gender_new
work_id,,,,
40,NaN,no-book,103685309,male
81,NaN,no-book,106806269,female
84,NaN,no-book,102279759,female
87,NaN,no-book,109405824,female
98,NaN,no-book,111175964,female
...,...,...,...,...
58370103,NaN,no-book,458370103,no-book-author
58376302,NaN,no-book,458376302,no-book-author
58376303,NaN,no-book,458376303,no-book-author
